In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from keras.models import Sequential 
from keras.layers.core import Dense, Dropout, Activation, Reshape
from keras.layers import concatenate
from keras.layers import Embedding
from keras import Input
import numpy as np
from keras.layers.recurrent import LSTM
from keras.layers import concatenate
from keras.layers.merge import Concatenate
from keras.models import model_from_json, Model
from keras.utils.vis_utils import plot_model
import h5py
import pickle
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [3]:
h5_que = h5py.File('/content/drive/MyDrive/VQA/Preprocessed Data/embedding_matrix_tokenize.h5', 'r')
embedding_matrix = h5_que['embedding_matrix_tokenize'][:]


In [5]:
image_model = Sequential()
#image_model.add(Reshape(input_shape = (4096,), target_shape=(4096,)))
image_model.add(Dense(1024, activation='relu',input_shape = (4096,)))

model1 = Model(inputs = image_model.input, outputs = image_model.output)
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_input (InputLayer)    [(None, 4096)]            0         
                                                                 
 dense (Dense)               (None, 1024)              4195328   
                                                                 
Total params: 4,195,328
Trainable params: 4,195,328
Non-trainable params: 0
_________________________________________________________________


In [6]:
language_model = Sequential()
language_model.add(Embedding( embedding_matrix.shape[0], embedding_matrix.shape[1], weights = [embedding_matrix], input_length=25, trainable = False,))
language_model.add(LSTM(512, return_sequences=False))
language_model.add(Dropout(0.5))
language_model.add(Dense(1024, activation='tanh'))
model2 = Model(language_model.input, language_model.output)
language_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 300)           4833000   
                                                                 
 lstm (LSTM)                 (None, 512)               1665024   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 1024)              525312    
                                                                 
Total params: 7,023,336
Trainable params: 2,190,336
Non-trainable params: 4,833,000
_________________________________________________________________


In [ ]:
combined = concatenate([image_model.output, language_model.output])
model = Dense(1024,  activation = 'tanh')(combined)
#model = Activation('tanh')(model)
model = Dropout(0.5)(model)

model = Dense(1001, activation = 'tanh')(model)
#model = Activation('tanh')(model)
model = Dropout(0.5)(model)

model = Dense(1001, activation = 'tanh')(model)
#model = Activation('tanh')(model)
model = Dropout(0.5)(model)

model = Dense(1001)(model)
model = Activation("softmax")(model)

model = Model(inputs=[image_model.input, language_model.input], outputs=model)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 embedding_input (InputLayer)   [(None, 25)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 25, 300)      4833000     ['embedding_input[0][0]']        
                                                                                                  
 lstm (LSTM)                    (None, 512)          1665024     ['embedding[0][0]']              
                                                                                                  
 dense_input (InputLayer)       [(None, 4096)]       0           []                               
                                                                                            

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='SGD',metrics=['accuracy'])

In [ ]:
%cd /content/drive/MyDrive/VQA/Preprocessed Data
import h5py
import numpy as np
h5_img = h5py.File('small_img_features_train.h5', 'r')
img_features_train = h5_img['small_img_features_train'][:]
image_train = np.repeat(img_features_train, 3, 0)

h5f = h5py.File('image_train.h5', 'w')
h5f.create_dataset('image_train', data=image_train)
h5f.close()

/content/drive/MyDrive/VQA/Preprocessed Data


In [ ]:
%cd /content/drive/MyDrive/VQA/Preprocessed Data
import h5py
import numpy as np
h5_img = h5py.File('small_img_features_val.h5', 'r')
img_features_train = h5_img['small_img_features_val'][:]
image_train = np.repeat(img_features_train, 3, 0)

h5f = h5py.File('image_train.val', 'w')
h5f.create_dataset('image_train', data=image_train)
h5f.close()

/content/drive/MyDrive/VQA/Preprocessed Data


In [ ]:
%cd /content/drive/MyDrive/VQA/Preprocessed Data
nb_epoch = 10
batch_size = 33  #99

h5_img = h5py.File('small_img_features_train.h5', 'r')
img_features_train = h5_img['small_img_features_train'][:]

h5_ans = h5py.File('small_answers_train.h5', 'r')
answer_train = h5_ans['small_answers_train'][:]

h5_que = h5py.File('small_question_train_tokenize.h5', 'r')
question_train = h5_que['small_question_train_tokenize'][:]

print("read the data")

print("epoch started!")
for e in range(nb_epoch):
  epoch_loss = []
  epoch_acc = []
  
  print("Epoch {}/{}".format((e+1), nb_epoch))
  nb_batch = 120000//batch_size
      
  for idy in range(nb_batch):
    start = idy * batch_size
    if idy == nb_batch - 1:
      end = 248349
    else:
      end = start + batch_size
  
    start_feat = start // 3
    end_feat = end // 3
    X_img = img_features_train[start_feat:end_feat,:]
    X_img = np.repeat(X_img, 3, 0)
    X_que = question_train[start:end,:]
    y     = answer_train[start:end,:]
    loss, acc = model.train_on_batch([X_img, X_que], y)
    
    epoch_loss.append(loss)
    epoch_acc.append(acc)
    
  print("cost: %f, acc: %f" % (loss, acc))
  print("cost average: %f, acc average: %f" % (np.mean(epoch_loss), np.mean(epoch_acc)))

model.save('model_10_with_32batchsize.h5')

/content/drive/MyDrive/VQA/Preprocessed Data
read the data
epoch started!
Epoch 1/10
cost: 2.625977, acc: 0.311111
cost average: 2.979126, acc average: 0.215199
Epoch 2/10
cost: 2.001055, acc: 0.422222
cost average: 2.497697, acc average: 0.275877
Epoch 3/10
cost: 1.794108, acc: 0.400000
cost average: 1.983389, acc average: 0.347486
Epoch 4/10
cost: 1.783020, acc: 0.466667
cost average: 1.864813, acc average: 0.364140
Epoch 5/10
cost: 1.782817, acc: 0.533333
cost average: 1.812190, acc average: 0.377268
Epoch 6/10
cost: 1.859124, acc: 0.444444
cost average: 1.766795, acc average: 0.389611
Epoch 7/10
cost: 1.839003, acc: 0.377778
cost average: 1.717165, acc average: 0.404128
Epoch 8/10
cost: 1.703564, acc: 0.488889
cost average: 1.667937, acc average: 0.419126
Epoch 9/10
cost: 1.659473, acc: 0.466667
cost average: 1.611532, acc average: 0.438364
Epoch 10/10
cost: 1.539523, acc: 0.577778
cost average: 1.550962, acc average: 0.456855


# Model tuning

In [ ]:
combined = concatenate([image_model.output, language_model.output])
model1 = Dense(1024,  activation = 'tanh')(combined)
#model = Activation('tanh')(model)
model1 = Dropout(0.5)(model1)

model1 = Dense(1001, activation = 'tanh')(model1)
#model = Activation('tanh')(model)
model1 = Dropout(0.5)(model1)

model1 = Dense(1001, activation = 'tanh')(model1)
#model = Activation('tanh')(model)
model1 = Dropout(0.5)(model1)

model1 = Dense(1001, activation = 'tanh')(model1)
#model = Activation('tanh')(model)
model1 = Dropout(0.5)(model1)

model1 = Dense(1001, activation = 'tanh')(model1)
#model = Activation('tanh')(model)
model1 = Dropout(0.5)(model1)

model1 = Dense(1001)(model1)
model1 = Activation("softmax")(model1)

model1 = Model(inputs=[image_model.input, language_model.input], outputs=model1)
model1.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 embedding_input (InputLayer)   [(None, 25)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 25, 300)      4833000     ['embedding_input[0][0]']        
                                                                                                  
 lstm (LSTM)                    (None, 512)          1665024     ['embedding[0][0]']              
                                                                                                  
 dense_input (InputLayer)       [(None, 4096)]       0           []                               
                                                                                            

In [ ]:
model1.compile(loss='categorical_crossentropy', optimizer='SGD',metrics=['accuracy'])

In [ ]:
%cd /content/drive/MyDrive/VQA/Preprocessed Data
nb_epoch = 5
batch_size = 33  #99

h5_img = h5py.File('small_img_features_train.h5', 'r')
img_features_train = h5_img['small_img_features_train'][:]

h5_ans = h5py.File('small_answers_train.h5', 'r')
answer_train = h5_ans['small_answers_train'][:]

h5_que = h5py.File('small_question_train_tokenize.h5', 'r')
question_train = h5_que['small_question_train_tokenize'][:]

print("read the data")

print("epoch started!")
for e in range(nb_epoch):
  epoch_loss = []
  epoch_acc = []
  
  print("Epoch {}/{}".format((e+1), nb_epoch))
  nb_batch = 120000//batch_size
      
  for idy in range(nb_batch):
    start = idy * batch_size
    if idy == nb_batch - 1:
      end = 248349
    else:
      end = start + batch_size
  
    start_feat = start // 3
    end_feat = end // 3
    X_img = img_features_train[start_feat:end_feat,:]
    X_img = np.repeat(X_img, 3, 0)
    X_que = question_train[start:end,:]
    y     = answer_train[start:end,:]
    loss, acc = model1.train_on_batch([X_img, X_que], y)
    
    epoch_loss.append(loss)
    epoch_acc.append(acc)
    
  print("cost: %f, acc: %f" % (loss, acc))
  print("cost average: %f, acc average: %f" % (np.mean(epoch_loss), np.mean(epoch_acc)))

model1.save('model1.h5')

/content/drive/MyDrive/VQA/Preprocessed Data
read the data
epoch started!
Epoch 1/5
cost: 2.580567, acc: 0.355556
cost average: 3.032529, acc average: 0.214269
Epoch 2/5
cost: 2.424647, acc: 0.244444
cost average: 2.876216, acc average: 0.224681
Epoch 3/5
cost: 2.491837, acc: 0.333333
cost average: 2.831385, acc average: 0.225948
Epoch 4/5
cost: 2.133488, acc: 0.288889
cost average: 2.364754, acc average: 0.290559
Epoch 5/5
cost: 1.894242, acc: 0.377778
cost average: 2.045572, acc average: 0.335229


In [7]:
combined = concatenate([image_model.output, language_model.output])
model1 = Dense(1024,  activation = 'relu')(combined)
#model = Activation('relu')(model)
model1 = Dropout(0.5)(model1)

model1 = Dense(1001, activation = 'relu')(model1)
#model = Activation('relu')(model)
model1 = Dropout(0.5)(model1)

model1 = Dense(1001, activation = 'relu')(model1)
#model = Activation('relu')(model)
model1 = Dropout(0.5)(model1)


model1 = Dense(1001)(model1)
model1 = Activation("softmax")(model1)

model1 = Model(inputs=[image_model.input, language_model.input], outputs=model1)
model1.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 embedding_input (InputLayer)   [(None, 25)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 25, 300)      4833000     ['embedding_input[0][0]']        
                                                                                                  
 lstm (LSTM)                    (None, 512)          1665024     ['embedding[0][0]']              
                                                                                                  
 dense_input (InputLayer)       [(None, 4096)]       0           []                               
                                                                                            

In [8]:
model1.compile(loss='categorical_crossentropy', optimizer='SGD',metrics=['accuracy'])

In [9]:
%cd /content/drive/MyDrive/VQA/Preprocessed Data
nb_epoch = 5
batch_size = 33  #99

h5_img = h5py.File('small_img_features_train.h5', 'r')
img_features_train = h5_img['small_img_features_train'][:]

h5_ans = h5py.File('small_answers_train.h5', 'r')
answer_train = h5_ans['small_answers_train'][:]

h5_que = h5py.File('small_question_train_tokenize.h5', 'r')
question_train = h5_que['small_question_train_tokenize'][:]

print("read the data")

print("epoch started!")
for e in range(nb_epoch):
  epoch_loss = []
  epoch_acc = []
  
  print("Epoch {}/{}".format((e+1), nb_epoch))
  nb_batch = 120000//batch_size
      
  for idy in range(nb_batch):
    start = idy * batch_size
    if idy == nb_batch - 1:
      end = 248349
    else:
      end = start + batch_size
  
    start_feat = start // 3
    end_feat = end // 3
    X_img = img_features_train[start_feat:end_feat,:]
    X_img = np.repeat(X_img, 3, 0)
    X_que = question_train[start:end,:]
    y     = answer_train[start:end,:]
    loss, acc = model1.train_on_batch([X_img, X_que], y)
    
    epoch_loss.append(loss)
    epoch_acc.append(acc)
    
  print("cost: %f, acc: %f" % (loss, acc))
  print("cost average: %f, acc average: %f" % (np.mean(epoch_loss), np.mean(epoch_acc)))

model1.save('model3.h5')

/content/drive/MyDrive/VQA/Preprocessed Data
read the data
epoch started!
Epoch 1/5
cost: 2.516427, acc: 0.311111
cost average: 2.944857, acc average: 0.224841
Epoch 2/5
cost: 2.122370, acc: 0.155556
cost average: 2.644186, acc average: 0.252826
Epoch 3/5
cost: 1.721853, acc: 0.288889
cost average: 2.092038, acc average: 0.324020
Epoch 4/5
cost: 1.743760, acc: 0.466667
cost average: 1.926470, acc average: 0.348780
Epoch 5/5
cost: 1.932231, acc: 0.355556
cost average: 1.857227, acc average: 0.360092


In [ ]:
combined = concatenate([image_model.output, language_model.output])
model1 = Dense(1024,  activation = 'sigmoid')(combined)
#model = Activation('sigmoid')(model)
model1 = Dropout(0.5)(model1)

model1 = Dense(1001, activation = 'sigmoid')(model1)
#model = Activation('relu')(model)
model1 = Dropout(0.5)(model1)

model1 = Dense(1001, activation = 'sigmoid')(model1)
#model = Activation('relu')(model)
model1 = Dropout(0.5)(model1)


model1 = Dense(1001)(model1)
model1 = Activation("softmax")(model1)

model1 = Model(inputs=[image_model.input, language_model.input], outputs=model1)
model1.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 embedding_input (InputLayer)   [(None, 25)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 25, 300)      4833000     ['embedding_input[0][0]']        
                                                                                                  
 lstm (LSTM)                    (None, 512)          1665024     ['embedding[0][0]']              
                                                                                                  
 dense_input (InputLayer)       [(None, 4096)]       0           []                               
                                                                                            

In [ ]:
model1.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
%cd /content/drive/MyDrive/VQA/Preprocessed Data
nb_epoch = 5
batch_size = 33  #99

h5_img = h5py.File('small_img_features_train.h5', 'r')
img_features_train = h5_img['small_img_features_train'][:]

h5_ans = h5py.File('small_answers_train.h5', 'r')
answer_train = h5_ans['small_answers_train'][:]

h5_que = h5py.File('small_question_train_tokenize.h5', 'r')
question_train = h5_que['small_question_train_tokenize'][:]

print("read the data")

print("epoch started!")
for e in range(nb_epoch):
  epoch_loss = []
  epoch_acc = []
  
  print("Epoch {}/{}".format((e+1), nb_epoch))
  nb_batch = 120000//batch_size
      
  for idy in range(nb_batch):
    start = idy * batch_size
    if idy == nb_batch - 1:
      end = 248349
    else:
      end = start + batch_size
  
    start_feat = start // 3
    end_feat = end // 3
    X_img = img_features_train[start_feat:end_feat,:]
    X_img = np.repeat(X_img, 3, 0)
    X_que = question_train[start:end,:]
    y     = answer_train[start:end,:]
    loss, acc = model1.train_on_batch([X_img, X_que], y)
    
    epoch_loss.append(loss)
    epoch_acc.append(acc)
    
  print("cost: %f, acc: %f" % (loss, acc))
  print("cost average: %f, acc average: %f" % (np.mean(epoch_loss), np.mean(epoch_acc)))

model1.save('model3.h5')

/content/drive/MyDrive/VQA/Preprocessed Data
read the data
epoch started!
Epoch 1/5




---



---



---



In [ ]:
%cd /content/drive/MyDrive/VQA/Preprocessed Data

h5_img = h5py.File('small_img_features_val.h5', 'r')
img_features_val = h5_img['small_img_features_val'][:]
img_features_val = np.repeat(img_features_val, 3, 0)
h5_ans = h5py.File('small_answers_val.h5', 'r')
answer_val = h5_ans['small_answers_val'][:]

h5_que = h5py.File('small_question_val_tokenize.h5', 'r')
question_val = h5_que['small_question_val_tokenize'][:]

/content/drive/MyDrive/VQA/Preprocessed Data


In [ ]:
model.evaluate([img_features_val,question_val],answer_val)

750/750 [==============================] - 19s 25ms/step - loss: 6.9240 - accuracy: 0.1168


[6.92402982711792, 0.11679166555404663]

In [ ]:
from tensorflow.keras.models import load_model
model1 = load_model('/content/drive/MyDrive/VQA/Preprocessed Data/model_1.h5')

In [ ]:
import numpy as np
from keras.layers import Embedding
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import keras.backend as K
import tensorflow as tf

In [ ]:
def extract_image_features(img_path):
  img = image.load_img(img_path,target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)
  x = preprocess_input(x)
  IMG_SHAPE = (224, 224, 3)
  model = tf.keras.applications.VGG16()


  last_layer = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('fc2').output)
  res = last_layer.predict(x)
  return res



In [ ]:
from keras.preprocessing.sequence import pad_sequences
import pickle 
questions = ["is this bird?"]
with open('/content/drive/MyDrive/Project dataset/Preprocessed Data/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)


train_question_tokenized = tokenizer.texts_to_sequences(questions)      
questions = pad_sequences(train_question_tokenized, maxlen = 25)          # len(X_train) x 30
questions

In [ ]:
x=[extract_image_features('/content/COCO_val2014_000000324670 (1).jpg'),questions]
historty=model1.predict(x)[0]


In [ ]:
import numpy as np
answers = np.argsort(historty[:1000])

In [ ]:
import operator
import pandas as pd
from collections import defaultdict
def int_to_answers():
	data_path = '/content/drive/MyDrive/VQA/Preprocessed Data/Training Data QA.pickle'
	df = pd.read_pickle(data_path)
	answers = df[['multiple_choice_answer']].values.tolist()
	freq = defaultdict(int)
	for answer in answers:
		freq[answer[0].lower()] += 1
	int_to_answer = sorted(freq.items(),key=operator.itemgetter(1),reverse=True)[0:1000]
	int_to_answer = [answer[0] for answer in int_to_answer]
	return int_to_answer

top_answers = int_to_answers()	

In [ ]:
top_answers[answers[-1]]